# Day 16 - Deepseek R1 Lite

In [1]:
import heapq

def find_min_cost(maze):
    # Find starting position 'S' and ending position 'E'
    start_x, start_y = -1, -1
    end_x, end_y = -1, -1
    for y in range(len(maze)):
        for x in range(len(maze[0])):
            if maze[y][x] == 'S':
                start_x, start_y = x, y
                maze[y][x] = '.'  # Replace 'S' with '.'
            elif maze[y][x] == 'E':
                end_x, end_y = x, y
                maze[y][x] = '.'  # Replace 'E' with '.'
    
    if start_x == -1 or end_x == -1:
        print("Start or end position not found.")
        return
    
    # Define directions and their movement vectors
    directions = ['East', 'North', 'West', 'South']
    dir_vectors = [(1, 0), (0, -1), (-1, 0), (0, 1)]
    
    # Initialize priority queue
    heap = []
    heapq.heappush(heap, (0, start_x, start_y, 0))  # cost, x, y, direction_index
    
    # Initialize minimum cost dictionary
    min_cost = {(start_x, start_y, 0): 0}
    
    while heap:
        current_cost, x, y, direction_index = heapq.heappop(heap)
        
        # If we've reached the end position, return the current cost
        if (x, y) == (end_x, end_y):
            print(current_cost)
            return
        
        # Move forward
        new_x = x + dir_vectors[direction_index][0]
        new_y = y + dir_vectors[direction_index][1]
        if 0 <= new_x < len(maze[0]) and 0 <= new_y < len(maze):
            if maze[new_y][new_x] != '#':
                new_state = (new_x, new_y, direction_index)
                new_cost = current_cost + 1
                if new_state not in min_cost or new_cost < min_cost[new_state]:
                    min_cost[new_state] = new_cost
                    heapq.heappush(heap, (new_cost, new_x, new_y, direction_index))
        
        # Rotate clockwise
        new_direction_index = (direction_index + 1) % 4
        new_state = (x, y, new_direction_index)
        new_cost = current_cost + 1000
        if new_state not in min_cost or new_cost < min_cost[new_state]:
            min_cost[new_state] = new_cost
            heapq.heappush(heap, (new_cost, x, y, new_direction_index))
        
        # Rotate counterclockwise
        new_direction_index = (direction_index - 1 + 4) % 4
        new_state = (x, y, new_direction_index)
        new_cost = current_cost + 1000
        if new_state not in min_cost or new_cost < min_cost[new_state]:
            min_cost[new_state] = new_cost
            heapq.heappush(heap, (new_cost, x, y, new_direction_index))
    
    # If the end was not reached
    print("No possible path.")

if __name__ == "__main__":
    with open('input.txt', 'r') as f:
        maze = [list(line.strip()) for line in f]
    find_min_cost(maze)

66404


## Part 2

In [2]:
import heapq

def read_maze(file_name):
    with open(file_name, 'r') as f:
        maze = [list(line.strip()) for line in f]
    return maze

def find_position(maze, target):
    for y in range(len(maze)):
        for x in range(len(maze[0])):
            if maze[y][x] == target:
                return x, y
    return -1, -1

def dijkstra(maze, start_states):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]
    heap = []
    for state in start_states:
        heapq.heappush(heap, (0, *state))
    cost = {state: 0 for state in start_states}
    
    while heap:
        current_cost, x, y, d = heapq.heappop(heap)
        if cost[(x, y, d)] < current_cost:
            continue
        for action in range(3):
            if action == 0:
                # Move forward
                new_x = x + directions[d][0]
                new_y = y + directions[d][1]
                new_d = d
                if 0 <= new_x < len(maze[0]) and 0 <= new_y < len(maze) and maze[new_y][new_x] != '#':
                    new_cost = current_cost + 1
                    if (new_x, new_y, new_d) not in cost or new_cost < cost[(new_x, new_y, new_d)]:
                        cost[(new_x, new_y, new_d)] = new_cost
                        heapq.heappush(heap, (new_cost, new_x, new_y, new_d))
            elif action == 1:
                # Rotate clockwise
                new_x = x
                new_y = y
                new_d = (d + 1) % 4
                new_cost = current_cost + 1000
                if (new_x, new_y, new_d) not in cost or new_cost < cost[(new_x, new_y, new_d)]:
                    cost[(new_x, new_y, new_d)] = new_cost
                    heapq.heappush(heap, (new_cost, new_x, new_y, new_d))
            elif action == 2:
                # Rotate counterclockwise
                new_x = x
                new_y = y
                new_d = (d - 1 + 4) % 4
                new_cost = current_cost + 1000
                if (new_x, new_y, new_d) not in cost or new_cost < cost[(new_x, new_y, new_d)]:
                    cost[(new_x, new_y, new_d)] = new_cost
                    heapq.heappush(heap, (new_cost, new_x, new_y, new_d))
    return cost

def main():
    maze = read_maze('input.txt')
    S_x, S_y = find_position(maze, 'S')
    E_x, E_y = find_position(maze, 'E')
    if S_x == -1 or E_x == -1:
        print("Start or end position not found.")
        return
    maze[S_y][S_x] = '.'
    maze[E_y][E_x] = '.'
    
    # Dijkstra from S
    start_S = [(S_x, S_y, 0)]
    cost_S = dijkstra(maze, start_S)
    
    # Dijkstra from E
    start_E = [(E_x, E_y, d) for d in range(4)]
    cost_E = dijkstra(maze, start_E)
    
    # Find total_min_cost
    total_min_cost = min(cost_S.get((E_x, E_y, d), float('inf')) for d in range(4))
    
    # Collect all non-wall tiles
    tiles = [(x, y) for y in range(len(maze)) for x in range(len(maze[0])) if maze[y][x] != '#']
    
    # Count tiles on best paths
    count = 0
    for x, y in tiles:
        for d in range(4):
            state = (x, y, d)
            if state in cost_S and state in cost_E:
                if cost_S[state] + cost_E[state] == total_min_cost:
                    count += 1
                    break  # No need to check other directions for this tile
    print(count)

if __name__ == "__main__":
    main()

79


In [3]:
import heapq

def read_maze(file_name):
    with open(file_name, 'r') as f:
        maze = [list(line.strip()) for line in f]
    return maze

def find_position(maze, target):
    for y in range(len(maze)):
        for x in range(len(maze[0])):
            if maze[y][x] == target:
                return x, y
    return -1, -1

def dijkstra(maze, start_states):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]
    heap = []
    for state in start_states:
        heapq.heappush(heap, (0, *state))
    cost = {state: 0 for state in start_states}
    
    while heap:
        current_cost, x, y, d = heapq.heappop(heap)
        if cost[(x, y, d)] < current_cost:
            continue
        # Move forward
        new_x = x + directions[d][0]
        new_y = y + directions[d][1]
        new_d = d
        if 0 <= new_x < len(maze[0]) and 0 <= new_y < len(maze) and maze[new_y][new_x] != '#':
            new_state = (new_x, new_y, new_d)
            new_cost = current_cost + 1
            if new_state not in cost or new_cost < cost[new_state]:
                cost[new_state] = new_cost
                heapq.heappush(heap, (new_cost, new_x, new_y, new_d))
        # Rotate clockwise
        new_d = (d + 1) % 4
        new_state = (x, y, new_d)
        new_cost = current_cost + 1000
        if new_state not in cost or new_cost < cost[new_state]:
            cost[new_state] = new_cost
            heapq.heappush(heap, (new_cost, x, y, new_d))
        # Rotate counter-clockwise
        new_d = (d - 1 + 4) % 4
        new_state = (x, y, new_d)
        new_cost = current_cost + 1000
        if new_state not in cost or new_cost < cost[new_state]:
            cost[new_state] = new_cost
            heapq.heappush(heap, (new_cost, x, y, new_d))
    return cost

def main():
    maze = read_maze('input.txt')
    S_x, S_y = find_position(maze, 'S')
    E_x, E_y = find_position(maze, 'E')
    if S_x == -1 or E_x == -1:
        print("Start or end position not found.")
        return
    maze[S_y][S_x] = '.'
    maze[E_y][E_x] = '.'
    
    # Dijkstra from S
    start_S = [(S_x, S_y, 0)]
    cost_S = dijkstra(maze, start_S)
    
    # Dijkstra from E
    start_E = [(E_x, E_y, d) for d in range(4)]
    cost_E = dijkstra(maze, start_E)
    
    # Find total minimal cost from S to E
    total_min_cost = float('inf')
    for d in range(4):
        if (E_x, E_y, d) in cost_S:
            total_min_cost = min(total_min_cost, cost_S[(E_x, E_y, d)])
    
    # Collect all non-wall tiles
    tiles = [(x, y) for y in range(len(maze)) for x in range(len(maze[0])) if maze[y][x] != '#']
    
    # Count tiles on best paths
    best_path_tiles = set()
    for x, y in tiles:
        for d in range(4):
            state = (x, y, d)
            if state in cost_S and state in cost_E:
                if cost_S[state] + cost_E[state] == total_min_cost:
                    best_path_tiles.add((x, y))
                    break  # No need to check other directions for this tile
    print(len(best_path_tiles))

if __name__ == "__main__":
    main()

79


In [4]:
import heapq

def read_maze(file_name):
    with open(file_name, 'r') as f:
        maze = [list(line.strip()) for line in f]
    return maze

def find_position(maze, target):
    for y in range(len(maze)):
        for x in range(len(maze[0])):
            if maze[y][x] == target:
                return x, y
    return -1, -1

def dijkstra(maze, start_states):
    directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]
    heap = []
    for state in start_states:
        heapq.heappush(heap, (0, *state))
    cost = {state: 0 for state in start_states}
    
    while heap:
        current_cost, x, y, d = heapq.heappop(heap)
        if cost[(x, y, d)] < current_cost:
            continue
        # Move forward
        new_x = x + directions[d][0]
        new_y = y + directions[d][1]
        new_d = d
        if 0 <= new_x < len(maze[0]) and 0 <= new_y < len(maze) and maze[new_y][new_x] != '#':
            new_state = (new_x, new_y, new_d)
            new_cost = current_cost + 1
            if new_state not in cost or new_cost < cost[new_state]:
                cost[new_state] = new_cost
                heapq.heappush(heap, (new_cost, new_x, new_y, new_d))
        # Rotate clockwise
        new_d = (d + 1) % 4
        new_state = (x, y, new_d)
        new_cost = current_cost + 1000
        if new_state not in cost or new_cost < cost[new_state]:
            cost[new_state] = new_cost
            heapq.heappush(heap, (new_cost, x, y, new_d))
        # Rotate counter-clockwise
        new_d = (d - 1 + 4) % 4
        new_state = (x, y, new_d)
        new_cost = current_cost + 1000
        if new_state not in cost or new_cost < cost[new_state]:
            cost[new_state] = new_cost
            heapq.heappush(heap, (new_cost, x, y, new_d))
    return cost

def main():
    maze = read_maze('input.txt')
    S_x, S_y = find_position(maze, 'S')
    E_x, E_y = find_position(maze, 'E')
    if S_x == -1 or E_x == -1:
        print("Start or end position not found.")
        return
    maze[S_y][S_x] = '.'
    maze[E_y][E_x] = '.'
    
    # Dijkstra from S
    start_S = [(S_x, S_y, 0)]
    cost_S = dijkstra(maze, start_S)
    
    # Dijkstra from E
    start_E = [(E_x, E_y, d) for d in range(4)]
    cost_E = dijkstra(maze, start_E)
    
    # Find total minimal cost from S to E
    total_min_cost = float('inf')
    for d in range(4):
        if (E_x, E_y, d) in cost_S:
            total_min_cost = min(total_min_cost, cost_S[(E_x, E_y, d)])
    
    # Collect all non-wall tiles
    tiles = [(x, y) for y in range(len(maze)) for x in range(len(maze[0])) if maze[y][x] != '#']
    
    # Count tiles on best paths
    best_path_tiles = set()
    for x, y in tiles:
        for d in range(4):
            state = (x, y, d)
            if state in cost_S and state in cost_E:
                if cost_S[state] + cost_E[state] == total_min_cost:
                    best_path_tiles.add((x, y))
                    break  # No need to check other directions for this tile
    print(len(best_path_tiles))

if __name__ == "__main__":
    main()

79


In [5]:
import heapq

def find_min_cost_forward(maze, start_x, start_y, end_x, end_y):
    directions = ['East', 'North', 'West', 'South']
    dir_vectors = [(1, 0), (0, -1), (-1, 0), (0, 1)]
    
    heap = []
    heapq.heappush(heap, (0, start_x, start_y, 0))  # cost, x, y, direction_index
    cost_forward = {}
    cost_forward[(start_x, start_y, 0)] = 0
    
    while heap:
        current_cost, x, y, direction = heapq.heappop(heap)
        if (x, y) == (end_x, end_y):
            continue  # Reached end, but continue to explore other paths
        
        if cost_forward.get((x, y, direction), float('inf')) < current_cost:
            continue  # Already found a better path to this state
        
        # Move forward
        new_x = x + dir_vectors[direction][0]
        new_y = y + dir_vectors[direction][1]
        if 0 <= new_x < len(maze[0]) and 0 <= new_y < len(maze) and maze[new_y][new_x] != '#':
            new_state = (new_x, new_y, direction)
            new_cost = current_cost + 1
            if new_cost < cost_forward.get(new_state, float('inf')):
                cost_forward[new_state] = new_cost
                heapq.heappush(heap, (new_cost, new_x, new_y, direction))
        
        # Rotate clockwise
        new_direction = (direction + 1) % 4
        new_state = (x, y, new_direction)
        new_cost = current_cost + 1000
        if new_cost < cost_forward.get(new_state, float('inf')):
            cost_forward[new_state] = new_cost
            heapq.heappush(heap, (new_cost, x, y, new_direction))
        
        # Rotate counter-clockwise
        new_direction = (direction - 1 + 4) % 4
        new_state = (x, y, new_direction)
        new_cost = current_cost + 1000
        if new_cost < cost_forward.get(new_state, float('inf')):
            cost_forward[new_state] = new_cost
            heapq.heappush(heap, (new_cost, x, y, new_direction))
    
    return cost_forward

def find_min_cost_backward(maze, start_x, start_y, end_x, end_y):
    directions = ['East', 'North', 'West', 'South']
    dir_vectors = [(1, 0), (0, -1), (-1, 0), (0, 1)]
    
    # Determine starting states for backward search
    start_states = []
    for d in range(4):
        opposite_d = (d + 2) % 4
        adj_x = end_x - dir_vectors[opposite_d][0]
        adj_y = end_y - dir_vectors[opposite_d][1]
        if 0 <= adj_x < len(maze[0]) and 0 <= adj_y < len(maze) and maze[adj_y][adj_x] != '#':
            start_states.append((adj_x, adj_y, d))
    
    heap = []
    for state in start_states:
        heapq.heappush(heap, (0, state[0], state[1], state[2]))
    
    cost_backward = {}
    for state in start_states:
        cost_backward[(state[0], state[1], state[2])] = 0
    
    while heap:
        current_cost, x, y, direction = heapq.heappop(heap)
        if (x, y) == (start_x, start_y):
            continue  # Reached start, but continue to explore other paths
        
        if cost_backward.get((x, y, direction), float('inf')) < current_cost:
            continue  # Already found a better path to this state
        
        # Move backward (which is forward in the opposite direction)
        opposite_d = (direction + 2) % 4
        new_x = x + dir_vectors[opposite_d][0]
        new_y = y + dir_vectors[opposite_d][1]
        if 0 <= new_x < len(maze[0]) and 0 <= new_y < len(maze) and maze[new_y][new_x] != '#':
            new_state = (new_x, new_y, opposite_d)
            new_cost = current_cost + 1
            if new_cost < cost_backward.get(new_state, float('inf')):
                cost_backward[new_state] = new_cost
                heapq.heappush(heap, (new_cost, new_x, new_y, opposite_d))
        
        # Rotate clockwise in backward search corresponds to rotating counter-clockwise in forward
        new_direction = (direction - 1 + 4) % 4
        new_state = (x, y, new_direction)
        new_cost = current_cost + 1000
        if new_cost < cost_backward.get(new_state, float('inf')):
            cost_backward[new_state] = new_cost
            heapq.heappush(heap, (new_cost, x, y, new_direction))
        
        # Rotate counter-clockwise in backward search corresponds to rotating clockwise in forward
        new_direction = (direction + 1) % 4
        new_state = (x, y, new_direction)
        new_cost = current_cost + 1000
        if new_cost < cost_backward.get(new_state, float('inf')):
            cost_backward[new_state] = new_cost
            heapq.heappush(heap, (new_cost, x, y, new_direction))
    
    return cost_backward

def main():
    with open('input.txt', 'r') as f:
        maze = [list(line.strip()) for line in f]
    
    # Find starting and ending positions
    start_x, start_y = -1, -1
    end_x, end_y = -1, -1
    for y in range(len(maze)):
        for x in range(len(maze[0])):
            if maze[y][x] == 'S':
                start_x, start_y = x, y
                maze[y][x] = '.'
            elif maze[y][x] == 'E':
                end_x, end_y = x, y
                maze[y][x] = '.'
    
    if start_x == -1 or end_x == -1:
        print("Start or end position not found.")
        return
    
    # Run forward Dijkstra's
    cost_forward = find_min_cost_forward(maze, start_x, start_y, end_x, end_y)
    
    # Find the minimum cost to reach 'E'
    C_min = float('inf')
    for d in range(4):
        if (end_x, end_y, d) in cost_forward:
            if cost_forward[(end_x, end_y, d)] < C_min:
                C_min = cost_forward[(end_x, end_y, d)]
    
    # Run backward Dijkstra's
    cost_backward = find_min_cost_backward(maze, start_x, start_y, end_x, end_y)
    
    # Collect positions that are part of some optimal path
    optimal_positions = set()
    for state, cost_f in cost_forward.items():
        if (state[0], state[1], state[2]) in cost_backward:
            cost_b = cost_backward[(state[0], state[1], state[2])]
            if cost_f + cost_b == C_min:
                optimal_positions.add((state[0], state[1]))
    
    # Ensure 'S' and 'E' are included
    optimal_positions.add((start_x, start_y))
    optimal_positions.add((end_x, end_y))
    
    # Count the number of unique positions
    print(len(optimal_positions))

if __name__ == "__main__":
    main()

2


In [6]:
import heapq

def find_min_cost_forward(maze, start_x, start_y, end_x, end_y):
    directions = ['East', 'North', 'West', 'South']
    dir_vectors = [(1, 0), (0, -1), (-1, 0), (0, 1)]
    
    heap = []
    heapq.heappush(heap, (0, start_x, start_y, 0))  # cost, x, y, direction_index
    cost_forward = {}
    cost_forward[(start_x, start_y, 0)] = 0
    
    while heap:
        current_cost, x, y, direction = heapq.heappop(heap)
        if (x, y) == (end_x, end_y):
            continue  # Reached end, but continue to explore other paths
        
        if cost_forward.get((x, y, direction), float('inf')) < current_cost:
            continue  # Already found a better path to this state
        
        # Move forward
        new_x = x + dir_vectors[direction][0]
        new_y = y + dir_vectors[direction][1]
        if 0 <= new_x < len(maze[0]) and 0 <= new_y < len(maze) and maze[new_y][new_x] != '#':
            new_state = (new_x, new_y, direction)
            new_cost = current_cost + 1
            if new_cost < cost_forward.get(new_state, float('inf')):
                cost_forward[new_state] = new_cost
                heapq.heappush(heap, (new_cost, new_x, new_y, direction))
        
        # Rotate clockwise
        new_direction = (direction + 1) % 4
        new_state = (x, y, new_direction)
        new_cost = current_cost + 1000
        if new_cost < cost_forward.get(new_state, float('inf')):
            cost_forward[new_state] = new_cost
            heapq.heappush(heap, (new_cost, x, y, new_direction))
        
        # Rotate counter-clockwise
        new_direction = (direction - 1 + 4) % 4
        new_state = (x, y, new_direction)
        new_cost = current_cost + 1000
        if new_cost < cost_forward.get(new_state, float('inf')):
            cost_forward[new_state] = new_cost
            heapq.heappush(heap, (new_cost, x, y, new_direction))
    
    return cost_forward

def find_min_cost_backward(maze, start_x, start_y, end_x, end_y):
    directions = ['East', 'North', 'West', 'South']
    dir_vectors = [(1, 0), (0, -1), (-1, 0), (0, 1)]
    
    # Determine starting states for backward search
    start_states = []
    for d in range(4):
        opposite_d = (d + 2) % 4
        adj_x = end_x - dir_vectors[opposite_d][0]
        adj_y = end_y - dir_vectors[opposite_d][1]
        if 0 <= adj_x < len(maze[0]) and 0 <= adj_y < len(maze) and maze[adj_y][adj_x] != '#':
            start_states.append((adj_x, adj_y, d))
    # Include 'E' with all directions
    for d in range(4):
        start_states.append((end_x, end_y, d))
    
    heap = []
    for state in start_states:
        heapq.heappush(heap, (0, state[0], state[1], state[2]))
    
    cost_backward = {}
    for state in start_states:
        cost_backward[(state[0], state[1], state[2])] = 0
    
    while heap:
        current_cost, x, y, direction = heapq.heappop(heap)
        if (x, y) == (start_x, start_y):
            continue  # Reached start, but continue to explore other paths
        
        if cost_backward.get((x, y, direction), float('inf')) < current_cost:
            continue  # Already found a better path to this state
        
        # Move backward (which is forward in the opposite direction)
        opposite_d = (direction + 2) % 4
        new_x = x + dir_vectors[opposite_d][0]
        new_y = y + dir_vectors[opposite_d][1]
        if 0 <= new_x < len(maze[0]) and 0 <= new_y < len(maze) and maze[new_y][new_x] != '#':
            new_state = (new_x, new_y, opposite_d)
            new_cost = current_cost + 1
            if new_cost < cost_backward.get(new_state, float('inf')):
                cost_backward[new_state] = new_cost
                heapq.heappush(heap, (new_cost, new_x, new_y, opposite_d))
        
        # Rotate clockwise in backward search corresponds to rotating counter-clockwise in forward
        new_direction = (direction - 1 + 4) % 4
        new_state = (x, y, new_direction)
        new_cost = current_cost + 1000
        if new_cost < cost_backward.get(new_state, float('inf')):
            cost_backward[new_state] = new_cost
            heapq.heappush(heap, (new_cost, x, y, new_direction))
        
        # Rotate counter-clockwise in backward search corresponds to rotating clockwise in forward
        new_direction = (direction + 1) % 4
        new_state = (x, y, new_direction)
        new_cost = current_cost + 1000
        if new_cost < cost_backward.get(new_state, float('inf')):
            cost_backward[new_state] = new_cost
            heapq.heappush(heap, (new_cost, x, y, new_direction))
    
    return cost_backward

def main():
    with open('input.txt', 'r') as f:
        maze = [list(line.strip()) for line in f]
    
    # Find starting and ending positions
    start_x, start_y = -1, -1
    end_x, end_y = -1, -1
    for y in range(len(maze)):
        for x in range(len(maze[0])):
            if maze[y][x] == 'S':
                start_x, start_y = x, y
                maze[y][x] = '.'
            elif maze[y][x] == 'E':
                end_x, end_y = x, y
                maze[y][x] = '.'
    
    if start_x == -1 or end_x == -1:
        print("Start or end position not found.")
        return
    
    # Run forward Dijkstra's
    cost_forward = find_min_cost_forward(maze, start_x, start_y, end_x, end_y)
    
    # Find the minimum cost to reach 'E'
    C_min = float('inf')
    for d in range(4):
        if (end_x, end_y, d) in cost_forward:
            if cost_forward[(end_x, end_y, d)] < C_min:
                C_min = cost_forward[(end_x, end_y, d)]
    
    # Run backward Dijkstra's
    cost_backward = find_min_cost_backward(maze, start_x, start_y, end_x, end_y)
    
    # Collect positions that are part of some optimal path
    optimal_positions = set()
    for (x, y, d), cost_f in cost_forward.items():
        if (x, y, d) in cost_backward:
            cost_b = cost_backward[(x, y, d)]
            if cost_f + cost_b == C_min:
                optimal_positions.add((x, y))
    
    # Ensure 'S' and 'E' are included
    optimal_positions.add((start_x, start_y))
    optimal_positions.add((end_x, end_y))
    
    # Count the number of unique positions
    print(len(optimal_positions))

if __name__ == "__main__":
    main()

2
